In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from datetime import datetime
import os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import requests
import json

s=requests.Session()
s.headers = {
    'authority': 'homes.hdb.gov.sg',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/plain, */*',
    'sessionid': 'a',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'content-type': 'application/json',
    'origin': 'https://homes.hdb.gov.sg',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://homes.hdb.gov.sg/home/finding-a-flat',
    'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
}


In [7]:

data = '{"location":"","coordinates":[["",""]],"range":"2","priceRangeLower":"0","priceRangeUpper":"0","flatType":"","waitingTime":"","modeOfSale":"BTO,SBF","floorRange":"","remainingLeaseRangeLower":1,"remainingLeaseRangeUpper":99,"ethnicGroup":"Chinese","rank":"Location, Price Range, Flat Type, Remaining Lease"}'

response = s.post('https://homes.hdb.gov.sg/home-api/public/v1/map/getCoordinatesByFilters', data=data)

response


<Response [200]>

In [8]:
jsonstr=json.loads(response.text)
detailedUrl='https://resources.homes.hdb.gov.sg/nf/{}.xml'
# https://resources.homes.hdb.gov.sg/nf/2021-05/bto/tg_d2c4/2021-05_BTO_VEdfRDJDNF8xNjE5NTAyODQxMDYz.xml
#all project details xml link follow this format - works well for BTO
for x in jsonstr:
    storageDict={}
#     to try and reduce API calls on price range
    x['properties']['description']=x['properties']['description'][0]
    if x['properties']['listingType']=='BTO':
        projPath=x['properties']['description'].get('photoUrl','').rpartition('/')[0]
        newUrl=detailedUrl.format(projPath+'/'+x['properties']['description']['projectId'])
    elif x['properties']['listingType']=='SBF':
        ballotQtr=x['properties']['description']['ballotQtr']
        projectID=x['properties']['description']['projectId']
        projDict='{"projectId":"'+projectID+'"}'
        storageDict={}
        x['availability']=json.loads(s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getProjectAvailabilityAndEthnic',data=projDict).text)
        for block,value in x['availability']['projectBlockFlatTypeInfoMap'].items():
            for flatType in value.keys():
                if storageDict.get(flatType) is None:
                    priceDict='{"projectId":"'+projectID+'","flatType":"'+flatType+'"}'
                    storageDict[flatType]=s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getPriceRangeAcrossTownByProjectIdAndFlatType',data=priceDict).text
                x['availability']['projectBlockFlatTypeInfoMap'][block][flatType]['areaPrice']=storageDict[flatType]
                    
        newUrl=detailedUrl.format(ballotQtr+'/sbf/unit_xml/UNIT_'+projectID)
    details=s.get(newUrl)
    x['details']=details.text

In [9]:
df=pd.json_normalize(jsonstr,max_level=1)
df.head(5)

,coordinates,details,properties.address,properties.listingType,properties.description,properties.hdbCategory,properties.region
0,"[1.3066321, 103.8852865]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Dakota Crest,BTO,{'photoUrl': '2022-02/bto/gl_n4c50/gl_n4c50_th...,1,CENTRAL REGION
1,"[1.3066965, 103.8592529]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",King George’s Heights,BTO,{'photoUrl': '2022-02/bto/kwn_n2c36/kwn_n2c36_...,1,CENTRAL REGION
2,"[1.3553976, 103.7291229]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Plantation Creek,BTO,{'photoUrl': '2022-02/bto/tg_d1c6/tg_d1c6_thum...,1,WEST REGION
3,"[1.3632007, 103.7317193]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Parc Flora @ Tengah,BTO,{'photoUrl': '2022-02/bto/tg_d3c5/tg_d3c5_thum...,1,WEST REGION
4,"[1.4336788, 103.8464212]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Yishun Boardwalk,BTO,{'photoUrl': '2022-02/bto/ys_n3c27/ys_n3c27_th...,1,NORTH REGION


In [10]:
df['details'][5]

'<?xml version="1.0" encoding="UTF-8"?>\n<template>\n    <project-name>Grove Spring @ Yishun</project-name>\n    <standard-project-info>\n        <key>\n            <town>Yishun</town>\n            <tooltip/>\n            <uri/>\n        </key>\n        <key>\n            <point>1524|917</point>\n        </key>\n        <key>\n            <latitude>1.4315820</latitude>\n            <longitude>103.8489962</longitude>\n        </key>\n        <key>\n            <completion-date>4Q2024</completion-date>\n            <tooltip>\n                \n        This is an estimated date that the flat will be completed and ready for key collection, and the actual completion date will depend on the progress of construction works on site. As the Estimated Completion Date (ECD) is indicative only, HDB shall not be responsible for any damage or loss arising from a change in the ECD.\n        &lt;br/>&lt;br/>\n        After booking the flat, buyers may check for updates via My HDBPage on HDB InfoWEB, as

In [11]:
# create past data save path
monthYear = datetime.now().strftime('%h-%Y').lower()
year = datetime.now().strftime('%Y')
savePath = os.path.join('past_data', year, monthYear)
Path(savePath).mkdir(parents=True, exist_ok=True)

print(savePath)

past_data\2022\feb-2022


In [12]:
# save data in various formats
fileName = os.path.join(savePath, 'hdb-sale-'+monthYear)
df.to_hdf(fileName+'.h5','HDBSALES')
df.to_pickle(fileName+'.pkl')
with open(fileName+".json", "w") as write_file:
    json.dump(jsonstr, write_file)

C:\Users\willi\miniconda3\envs\workspace\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['coordinates', 'details', 'properties.address',
       'properties.listingType', 'properties.description',
       'properties.hdbCategory', 'properties.region'],
      dtype='object')]

  pytables.to_hdf(


In [13]:
jsonstr[1]

{'coordinates': '[1.3066965, 103.8592529]',
 'properties': {'address': 'King George’s Heights',
  'listingType': 'BTO',
  'description': {'photoUrl': '2022-02/bto/kwn_n2c36/kwn_n2c36_thumbnail.jpg',
   'maxRemainingLease': '99',
   'stage': 'Launch',
   'distance': '-1',
   'town': 'Kallang/Whampoa',
   'launchStartDate': '2022-02-17 10:00:00.0',
   'ballotQtr': '2022-02',
   'flatType': '3-Room (Income Ceiling $14,000), 4-Room',
   'maxPrice': '675000',
   'projectId': '2022-02_BTO_S1dOX04yQzM2XzE2NDI5ODg3MDcwMzQ'},
  'hdbCategory': '1',
  'region': 'CENTRAL REGION'},
 'details': '<?xml version="1.0" encoding="UTF-8"?>\n<template>\n    <project-name>King George’s Heights</project-name>\n    <standard-project-info>\n        <key>\n            <town>Kallang/Whampoa</town>\n            <tooltip/>\n            <uri/>\n        </key>\n        <key>\n            <point>1584|1553</point>\n        </key>\n        <key>\n            <latitude>1.3066965</latitude>\n            <longitude>103.85